In [45]:
!python -m spacy download en_core_web_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     --------- ------------------------------ 2.9/12.8 MB 18.6 MB/s eta 0:00:01
     ----------------- ---------------------- 5.5/12.8 MB 15.2 MB/s eta 0:00:01
     ------------------------- -------------- 8.1/12.8 MB 14.0 MB/s eta 0:00:01
     ----------------------------- ---------- 9.4/12.8 MB 11.7 MB/s eta 0:00:01
     ------------------------------------- - 12.3/12.8 MB 12.4 MB/s eta 0:00:01
     --------------------------------------- 12.8/12.8 MB 11.8 MB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [2]:
import spacy
import os
import re

In [3]:
nlp = spacy.load("en_core_web_sm")

C:\Users\santhoshs.s\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\santhoshs.s\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\utils\generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
C:\Users\santhoshs.s\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\utils\generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [4]:
def extract_information(resume_text):
  """Extracts information from resume text using spaCy."""

  doc = nlp(resume_text)

  name = None
  email = None
  phone = None
  skills = []
  experience = []
  education = []

  for entity in doc.ents:
    if entity.label_ == "PERSON":
      name = entity.text
    elif entity.label_ == "EMAIL":
      email = entity.text
    elif entity.label_ == "ORG":
      experience.append(entity.text)
    elif entity.label_ == "GPE":
      education.append(entity.text)

  for token in doc:
      if token.pos_ == "NOUN" or token.pos_ == "PROPN":
          if token.text.lower() in ['python','java','c++','sql']:
              skills.append(token.text)
  
  # Extract phone number using regex
  phone_match = re.findall(r'\(?\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}', resume_text)
  if phone_match:
    phone = phone_match[0]

  return {
      "name": name,
      "email": email,
      "phone": phone,
      "skills": skills,
      "experience": experience,
      "education": education,
  }

In [5]:
import pandas as pd

In [14]:
extracted_info_list = []
def process_resumes_in_folder(folder_path):
      # Initialize a list to store extracted information

    # Your existing code to process resumes and extract information
    # For example:
    for resume_file in os.listdir(folder_path):
        resume_path = os.path.join(folder_path, resume_file)
        # Assuming you have a function to extract info from the resume
        extracted_info = extract_information(resume_path)  # This should return a dictionary
        if extracted_info:  # Check if extracted_info is not None or empty
            extracted_info_list.append(extracted_info)  # Append the extracted info to the list

    return extracted_info_list  # Return the list of extracted information


In [15]:
folder_path = "C:\\Users\\santhoshs.s\\jupyter\\resumes\\data\\data\\BPO"
process_resumes_in_folder(folder_path)

[{'name': None,
  'email': None,
  'phone': None,
  'skills': [],
  'experience': [],
  'education': []},
 {'name': None,
  'email': None,
  'phone': None,
  'skills': [],
  'experience': [],
  'education': []},
 {'name': None,
  'email': None,
  'phone': None,
  'skills': [],
  'experience': [],
  'education': []},
 {'name': None,
  'email': None,
  'phone': None,
  'skills': [],
  'experience': [],
  'education': []},
 {'name': None,
  'email': None,
  'phone': None,
  'skills': [],
  'experience': [],
  'education': []},
 {'name': None,
  'email': None,
  'phone': None,
  'skills': [],
  'experience': [],
  'education': []},
 {'name': None,
  'email': None,
  'phone': None,
  'skills': [],
  'experience': [],
  'education': []},
 {'name': None,
  'email': None,
  'phone': None,
  'skills': [],
  'experience': [],
  'education': []},
 {'name': None,
  'email': None,
  'phone': None,
  'skills': [],
  'experience': [],
  'education': []},
 {'name': None,
  'email': None,
  'phone': No

In [16]:
import csv

def save_to_csv(extracted_info_list, filename="extracted_resume_info.csv"):
    """Saves the extracted information to a CSV file."""
    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['name', 'email', 'phone', 'skills', 'experience', 'education']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for extracted_info in extracted_info_list:
            writer.writerow(extracted_info)

In [17]:
save_to_csv(extracted_info_list)

In [6]:
import fitz  # PyMuPDF for reading PDFs
import re
import pandas as pd
import os
import nltk
from nltk.tokenize import word_tokenize

# Function to extract text from PDF file and convert it to lowercase
def extract_text_from_pdf(pdf_path):
    with fitz.open(pdf_path) as doc:
        text = ""
        for page in doc:
            text += page.get_text()
    return text.lower()  # Convert the entire text to lowercase

# Function to extract the candidate's name
def extract_name(text):
    name_patterns = [
        r"[A-Z][a-z]+ [A-Z][a-z]+",  # Two words with capital letters (first name + last name)
        r"[A-Z][a-z]+-\w+"  # Hyphenated names like "Anne-Marie"
    ]

    for pattern in name_patterns:
        match = re.search(pattern, text, re.IGNORECASE)  # Case-insensitive matching
        if match:
            return match.group()

    # If regular expressions fail, try using NLTK for Named Entity Recognition (NER)
    try:
        tokens = word_tokenize(text)
        tagged_text = nltk.pos_tag(tokens)
        for word, tag in tagged_text:
            if tag == 'NNP':  # Proper noun, typically used for names
                return word
    except LookupError:
        print("NLTK data not downloaded. Install NLTK and run 'nltk.download()' to download required resources.")

    return "Not Found"

# Function to extract the profile summary
import re

def extract_profile_summary(text):
    # List of potential keywords indicating the profile section
    keywords = [
        "summary", "profile", "objective", "professional summary",
        "career summary", "executive summary", "personal summary",
        "summary of qualifications", "overview"
    ]
    
    # Convert the text to lowercase for case-insensitive matching
    lines = text.splitlines()  # Split the text into lines
    summary_start = None

    # Normalize keywords: Create regex patterns to ignore spaces
    keyword_patterns = [re.compile(r'\s*'.join(list(keyword.lower()))) for keyword in keywords]

    # Attempt to find the starting point of the summary based on keywords
    for pattern in keyword_patterns:
        for idx, line in enumerate(lines):
            if pattern.search(line.lower()):  # Check for keyword with regex
                summary_start = idx
                break
        if summary_start is not None:
            break

    # If a summary section was found, extract subsequent lines
    if summary_start is not None:
        extracted_lines = []
        
        # Start extracting from the next line after the keyword
        for i in range(summary_start + 1, len(lines)):
            line = lines[i].strip()

            # Stop extraction if the line is empty or contains bullet points, but allow for continuation
            if line == "" or line.startswith("*") or line.startswith("#"):
                continue  # Allow extraction to continue past bullet points and empty lines
            if line.isupper():  # Stop if we reach a fully uppercase line, which may indicate a section title
                break
            
            # Add the line to the extracted lines
            extracted_lines.append(line)

        # Join the extracted lines into a single summary string
        summary_result = "\n".join(extracted_lines).strip()
        return summary_result if summary_result else "Not Found"
    
    return "Not Found"

# Function to extract email address
def extract_email(text):
    email_pattern = r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}'
    match = re.search(email_pattern, text)
    return match.group(0) if match else "Not Found"

# Function to extract the phone number
def extract_phone_number(text):
    # Regular expression pattern for phone numbers, allowing symbols like (), -, and spaces
    phone_pattern = r'\b(?:\+?(\d{1,3}))?[-. ()]*(\d{3})[-. )]*(\d{3})[-. ]*(\d{4})\b'
    match = re.search(phone_pattern, text)
    
    # If a match is found, remove all non-digit characters to get a clean number
    if match:
        # Combine all groups into a single string and remove any non-numeric characters
        raw_number = match.group(0)
        clean_number = re.sub(r'\D', '', raw_number)  # Remove all non-digit characters
        return clean_number
    
    return "Not Found"


# Function to extract address without including the keyword "address"
def extract_address(text):
    address_keywords = ['address', 'street', 'city', 'state', 'zip', 'postal']
    lines = text.splitlines()
    
    # Iterate through the lines to find keywords
    for idx, line in enumerate(lines):
        # Check if any of the address keywords are in the line
        if any(keyword in line.lower() for keyword in address_keywords):
            # Extract lines after the keyword to capture the full address
            extracted_lines = []
            
            # If the keyword "address" is present in the line, ignore it and start from the next part of the line
            clean_line = re.sub(r'\b(?:' + '|'.join(address_keywords) + r')\b', '', line, flags=re.IGNORECASE).strip()
            if clean_line:
                extracted_lines.append(clean_line)
            
            # Attempt to include a few subsequent lines in case the address continues
            for i in range(1, 3):
                if idx + i < len(lines):
                    next_line = lines[idx + i].strip()
                    if next_line:
                        extracted_lines.append(next_line)
            
            # Combine the extracted lines into a single string
            return " ".join(extracted_lines).strip()
    
    return "Not Found"

import re
def extract_links(text):
    """
    Extracts hyperlinks from text, including standard URLs, HTML-like links, and markdown links.
    
    Args:
        text (str): The text from which to extract links.
        
    Returns:
        list: A list of extracted hyperlinks or "Not Found" if no links are found.
    """
    # Standard URLs
    link_pattern = r'https?://[^\s]+'
    
    # HTML links
    html_link_pattern = r'<a\s+(?:[^>]*?\s+)?href=["\'](https?://[^\s"\']+)["\']'
    
    # Markdown links
    markdown_link_pattern = r'\[.*?\]\((https?://[^\s]+)\)'
    
    # Plain links like www.example.com or example.com
    plain_link_pattern = r'\b(?:www\.)?[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}(?:/[^\s]*)?\b'
    
    # Find all matches for each pattern
    standard_links = re.findall(link_pattern, text)
    html_links = re.findall(html_link_pattern, text)
    markdown_links = re.findall(markdown_link_pattern, text)
    plain_links = re.findall(plain_link_pattern, text)
    
    # Combine all found links into a single list
    hyperlinks = standard_links + html_links + markdown_links + plain_links
    
    # Remove duplicates by converting to a set, then back to a list
    unique_links = list(set(hyperlinks))
    
    return unique_links if unique_links else ["Not Found"]

def extract_experience(text):
    """
    Extracts work experience from the resume text.

    Args:
        text (str): The text from which to extract work experience.

    Returns:
        list: A list of dictionaries containing job title, company, location, dates, and responsibilities.
    """
    experience_list = []
    
    # Improved regex to capture common experience formats
    experience_pattern = r'(?i)(.+?)(?:\s*-\s*|\s*\(?\s*(?:\w+\s*)+\s*\)?)\s*(.+?)\s*[,|]\s*(.+?)\s*[-–]?\s*(\d{4}[-–]?\d{0,4})?\s*[-–]?\s*(.*?)(?=\n\s*\n|\Z)'

    # Find all matches
    matches = re.findall(experience_pattern, text, re.DOTALL)

    for match in matches:
        job_title = match[0].strip() if match[0] else "Not Found"
        company = match[1].strip() if match[1] else "Not Found"
        location = match[2].strip() if match[2] else "Not Found"
        employment_dates = match[3].strip() if match[3] else "Not Found"
        responsibilities = match[4].strip().replace('\n', ' ') if match[4] else "Not Found"

        experience_list.append({
            'Job Title': job_title,
            'Company': company,
            'Location': location,
            'Employment Dates': employment_dates,
            'Responsibilities': responsibilities
        })

    return experience_list if experience_list else [{"Experience": "Not Found"}]

# Function to extract education details
def extract_education(text):
    education_list = []
    education_keywords = ["education", "academic background"]
    for keyword in education_keywords:
        education_start = text.find(keyword)
        if education_start != -1:
            lines = text.splitlines()[education_start:]  # Text after education section
            for line in lines:
                if line.strip() == "":
                    continue  # Skip empty lines
                university_match = re.search(r"[A-Z][a-z]+(?:\s[A-Z][a-z]+)?", line, re.IGNORECASE)
                if university_match:
                    university = university_match.group()
                    degree_match = re.search(r"[A-Z][a-z]+(?:\s[A-Z][a-z]+)?", line, re.IGNORECASE)
                    degree = degree_match.group() if degree_match else ""
                    year_match = re.search(r"\d{4}", line)
                    year = year_match.group() if year_match else ""
                    education_list.append({
                        "University": university,
                        "Degree": degree,
                        "Year": year
                    })
            break
    return education_list

# Function to extract languages
def extract_languages(text):
    language_keywords = ["languages", "skills", "proficiency"]
    for keyword in language_keywords:
        language_start = text.find(keyword)
        if language_start != -1:
            lines = text.splitlines()[language_start:]
            languages = []
            for line in lines:
                if line.strip() == "":
                    continue
                language_match = re.search(r"[A-Z][a-z]+(?:\s[A-Z][a-z]+)?", line, re.IGNORECASE)
                if language_match:
                    languages.append(language_match.group())
            return languages
    return []

# Function to extract certificates
def extract_certificates(text):
    certificate_keywords = ["certifications", "accreditations"]
    for keyword in certificate_keywords:
        certificate_start = text.find(keyword)
        if certificate_start != -1:
            lines = text.splitlines()[certificate_start:]
            certificates = []
            for line in lines:
                if line.strip() == "":
                    continue
                certificate_match = re.search(r"[A-Z][a-z]+(?:\s[A-Z][a-z]+)?", line, re.IGNORECASE)
                if certificate_match:
                    certificates.append(certificate_match.group())
            return certificates
    return []

# Function to process all PDFs in a folder and save the extracted info in a CSV
import os
import pandas as pd

# Function to process all PDFs in a folder and save the extracted info in a CSV
def process_resumes(folder_path, output_csv_path):
    # Create an empty DataFrame to hold new data
    data = []

    # Loop through all files in the specified folder
    for filename in os.listdir(folder_path):
        if filename.endswith('.pdf'):
            pdf_path = os.path.join(folder_path, filename)
            text = extract_text_from_pdf(pdf_path)

            # Extract information (profile, name, email, phone, address, links, experience, education, languages, certificates)
            name = extract_name(text)
            email = extract_email(text)
            phone = extract_phone_number(text)
            address = extract_address(text)
            links = extract_links(text)  # Use the modified links extraction function
            experience = extract_experience(text)
            education = extract_education(text)
            languages = extract_languages(text)
            certificates = extract_certificates(text)
            profile_summary = extract_profile_summary(text)

            # Add extracted info to data list
            data.append({
                'File Name': filename,
                'Name': name,
                'Email': email,
                'Phone': phone,
                'Address': address,
                'Links': ', '.join(links),  # Convert list of links to a comma-separated string
                'Experience': experience,
                'Education': education,
                'Languages': languages,
                'Certificates': certificates,
                'Profile Summary': profile_summary
            })

    # Create a DataFrame from the collected data
    df = pd.DataFrame(data)

    # Clear previous data in the CSV file
    if os.path.exists(output_csv_path):
        os.remove(output_csv_path)  # Remove the existing file

    # Save the new data to the CSV file
    df.to_csv(output_csv_path, index=False)
    print(f'Data saved to {output_csv_path}')



# Example usage:
# Replace 'path_to_resume_folder' with the actual folder path containing PDF resumes
# Replace 'output.csv' with your desired output file name
process_resumes('C:\\Users\\santhoshs.s\\jupyter\\resumes\\data\\data\\BPO\\bb', 'output2.csv')

Data saved to output2.csv


In [ ]:
# Sample testing code
if __name__ == "__main__":
    # Sample resume text for testing
    sample_text = """
    Work Experience

    Nutrition Consultant - WIC (Port Washington) 2021 - Present
    - Provided nutrition education counseling on morbid obesity, high cholesterol, and diabetes.

    Nutrition Consultant - DaVita (Delmas) 2016 - 2019
    - Registered nutrition consultant/educator responsible for educating, counseling, and supporting patients.

    Education
    Master of Science in Dietary Education, Golden Valley University
    """
    
    extracted_experience = extract_experience(sample_text)
    print(extracted_experience)


In [1]:
import fitz  # PyMuPDF for reading PDFs
import re
import pandas as pd
import os
import nltk
from nltk.tokenize import word_tokenize

# Function to extract text from PDF file and convert it to lowercase
def extract_text_from_pdf(pdf_path):
    with fitz.open(pdf_path) as doc:
        text = ""
        for page in doc:
            text += page.get_text()
    return text.lower()  # Convert the entire text to lowercase

# Function to extract the candidate's name
def extract_name(text):
    name_patterns = [
        r"[A-Z][a-z]+ [A-Z][a-z]+",  # Two words with capital letters (first name + last name)
        r"[A-Z][a-z]+-\w+"  # Hyphenated names like "Anne-Marie"
    ]

    for pattern in name_patterns:
        match = re.search(pattern, text, re.IGNORECASE)  # Case-insensitive matching
        if match:
            return match.group()

    # If regular expressions fail, try using NLTK for Named Entity Recognition (NER)
    try:
        tokens = word_tokenize(text)
        tagged_text = nltk.pos_tag(tokens)
        for word, tag in tagged_text:
            if tag == 'NNP':  # Proper noun, typically used for names
                return word
    except LookupError:
        print("NLTK data not downloaded. Install NLTK and run 'nltk.download()' to download required resources.")

    return "Not Found"

# Function to extract the profile summary
def extract_profile_summary(text):
    # List of potential keywords indicating the profile section
    keywords = [
        "summary", "profile", "objective", "professional summary",
        "career summary", "executive summary", "personal summary",
        "summary of qualifications", "overview"
    ]
    
    # Convert the text to lowercase for case-insensitive matching
    lines = text.splitlines()  # Split the text into lines
    summary_start = None

    # Normalize keywords: Create regex patterns to ignore spaces
    keyword_patterns = [re.compile(r'\s*'.join(list(keyword.lower()))) for keyword in keywords]

    # Attempt to find the starting point of the summary based on keywords
    for pattern in keyword_patterns:
        for idx, line in enumerate(lines):
            if pattern.search(line.lower()):  # Check for keyword with regex
                summary_start = idx
                break
        if summary_start is not None:
            break

    # If a summary section was found, extract subsequent lines
    if summary_start is not None:
        extracted_lines = []
        
        # Start extracting from the next line after the keyword
        for i in range(summary_start + 1, len(lines)):
            line = lines[i].strip()

            # Stop extraction if the line is empty or contains bullet points, but allow for continuation
            if line == "" or line.startswith("*") or line.startswith("#"):
                continue  # Allow extraction to continue past bullet points and empty lines
            if line.isupper():  # Stop if we reach a fully uppercase line, which may indicate a section title
                break
            
            # Add the line to the extracted lines
            extracted_lines.append(line)

        # Join the extracted lines into a single summary string
        summary_result = "\n".join(extracted_lines).strip()
        return summary_result if summary_result else "Not Found"
    
    return "Not Found"

# Function to extract email address
def extract_email(text):
    email_pattern = r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}'
    match = re.search(email_pattern, text)
    return match.group(0) if match else "Not Found"

# Function to extract the phone number
def extract_phone_number(text):
    phone_pattern = r'\b(?:\+?(\d{1,3}))?[-. (]*(\d{3})[-. )]*(\d{3})[-. ]*(\d{4})\b'
    match = re.search(phone_pattern, text)
    return match.group(0).replace("(", "").replace(")", "").replace("-", "").replace(" ", "") if match else "Not Found"

# Function to extract address without including the keyword "address"
def extract_address(text):
    address_keywords = ['address', 'street', 'city', 'state', 'zip', 'postal']
    lines = text.splitlines()
    
    # Iterate through the lines to find keywords
    for idx, line in enumerate(lines):
        # Check if any of the address keywords are in the line
        if any(keyword in line.lower() for keyword in address_keywords):
            # Extract lines after the keyword to capture the full address
            extracted_lines = []
            
            # If the keyword "address" is present in the line, ignore it and start from the next part of the line
            clean_line = re.sub(r'\b(?:' + '|'.join(address_keywords) + r')\b', '', line, flags=re.IGNORECASE).strip()
            if clean_line:
                extracted_lines.append(clean_line)
            
            # Attempt to include a few subsequent lines in case the address continues
            for i in range(1, 3):
                if idx + i < len(lines):
                    next_line = lines[idx + i].strip()
                    if next_line:
                        extracted_lines.append(next_line)
            
            # Combine the extracted lines into a single string
            return " ".join(extracted_lines).strip()
    
    return "Not Found"

# Function to extract links
def extract_links(text):
    """
    Extracts hyperlinks from text, including standard URLs, HTML-like links, and markdown links.
    
    Args:
        text (str): The text from which to extract links.
        
    Returns:
        list: A list of extracted hyperlinks or "Not Found" if no links are found.
    """
    # Standard URLs
    link_pattern = r'https?://[^\s]+'
    
    # HTML links
    html_link_pattern = r'<a\s+(?:[^>]*?\s+)?href=["\'](https?://[^\s"\']+)["\']'
    
    # Markdown links
    markdown_link_pattern = r'\[.*?\]\((https?://[^\s]+)\)'
    
    # Plain links like www.example.com or example.com
    plain_link_pattern = r'\b(?:www\.)?[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}(?:/[^\s]*)?\b'
    
    # Find all matches for each pattern
    standard_links = re.findall(link_pattern, text)
    html_links = re.findall(html_link_pattern, text)
    markdown_links = re.findall(markdown_link_pattern, text)
    plain_links = re.findall(plain_link_pattern, text)
    
    # Combine all found links into a single list
    hyperlinks = standard_links + html_links + markdown_links + plain_links
    
    # Remove duplicates by converting to a set, then back to a list
    unique_links = list(set(hyperlinks))
    
    return unique_links if unique_links else ["Not Found"]

# Function to extract experience details
def extract_experience(text):
    experience_list = []

    # List of keywords that indicate the start of an experience section
    experience_keywords = [
        "work history", "employment history", "work experience", "employment experience",
        "employment", "professional experience", "professional background",
        "military background", "military experience", "internships",
        "paid internships", "work internships", "professional internships",
        "employment background"
    ]

    # Create a regex pattern for the keywords (case insensitive)
    keywords_pattern = r'(?i)(' + '|'.join(experience_keywords) + r')'

    # Find the starting point for experience extraction
    experience_start = re.search(keywords_pattern, text)
    if experience_start:
        # Extract text starting from the found keyword
        experience_section = text[experience_start.end():]

        # Improved regex pattern to capture job title, company, location, dates, and responsibilities
        experience_pattern = r'(?i)(?P<job_title>.+?)(?:\s+at\s+|\s*-*\s*)(?P<company>[^\n,]+?)\s*,?\s*(?P<location>[^\n,]+?)\s*[-–]?\s*(?P<dates>[\w\s]+(?:[-–]\s*[\w\s]+)?)?\s*(?P<responsibilities>.+?)(?=\n\s*\n|\Z)'

        # Find all matches in the extracted experience section
        matches = re.finditer(experience_pattern, experience_section, re.DOTALL)

        for match in matches:
            job_title = match.group("job_title").strip() if match.group("job_title") else "Not Found"
            company = match.group("company").strip() if match.group("company") else "Not Found"
            location = match.group("location").strip() if match.group("location") else "Not Found"
            employment_dates = match.group("dates").strip() if match.group("dates") else "Not Found"
            responsibilities = match.group("responsibilities").strip().replace('\n', ' ') if match.group("responsibilities") else "Not Found"

            experience_list.append({
                'Job Title': job_title,
                'Company': company,
                'Location': location,
                'Employment Dates': employment_dates,
                'Responsibilities': responsibilities
            })

    return experience_list if experience_list else [{"Experience": "Not Found"}]



# Function to extract education details
def extract_education(text):
    education_list = []
    education_keywords = ["education", "academic background"]
    for keyword in education_keywords:
        education_start = text.find(keyword)
        if education_start != -1:
            lines = text.splitlines()[education_start:]  # Text after education section
            for line in lines:
                if line.strip() == "":
                    continue  # Skip empty lines
                university_match = re.search(r"[A-Z][a-z]+(?:\s[A-Z][a-z]+)?", line, re.IGNORECASE)
                if university_match:
                    university = university_match.group()
                    degree_match = re.search(r"[A-Z][a-z]+(?:\s[A-Z][a-z]+)?", line, re.IGNORECASE)
                    degree = degree_match.group() if degree_match else ""
                    year_match = re.search(r"\d{4}", line)
                    year = year_match.group() if year_match else ""
                    education_list.append({
                        "University": university,
                        "Degree": degree,
                        "Year": year
                    })
            break
    return education_list

# Function to extract languages
def extract_languages(text):
    language_keywords = ["languages", "skills", "proficiency"]
    for keyword in language_keywords:
        language_start = text.find(keyword)
        if language_start != -1:
            lines = text.splitlines()[language_start:]
            languages = []
            for line in lines:
                if line.strip() == "":
                    continue
                language_match = re.search(r"[A-Z][a-z]+(?:\s[A-Z][a-z]+)?", line, re.IGNORECASE)
                if language_match:
                    languages.append(language_match.group())
            return languages
    return []

# Function to extract certificates
def extract_certificates(text):
    certificate_keywords = ["certifications", "accreditations"]
    for keyword in certificate_keywords:
        certificate_start = text.find(keyword)
        if certificate_start != -1:
            lines = text.splitlines()[certificate_start:]
            certificates = []
            for line in lines:
                if line.strip() == "":
                    continue
                certificate_match = re.search(r"[A-Z][a-z]+(?:\s[A-Z][a-z]+)?", line, re.IGNORECASE)
                if certificate_match:
                    certificates.append(certificate_match.group())
            return certificates
    return []

# Function to process all PDFs in a folder and save the extracted info in a CSV
def process_resumes(folder_path, output_csv_path):
    # Create an empty DataFrame to hold new data
    data = []

    # Loop through all files in the specified folder
    for filename in os.listdir(folder_path):
        if filename.endswith('.pdf'):
            pdf_path = os.path.join(folder_path, filename)
            text = extract_text_from_pdf(pdf_path)

            # Extract information (profile, name, email, phone, address, links, experience, education, languages, certificates)
            name = extract_name(text)
            profile_summary = extract_profile_summary(text)
            email = extract_email(text)
            phone = extract_phone_number(text)
            address = extract_address(text)
            links = extract_links(text)
            experience = extract_experience(text)
            education = extract_education(text)
            languages = extract_languages(text)
            certificates = extract_certificates(text)

            # Append extracted data to the list
            data.append({
                'Name': name,
                'Profile Summary': profile_summary,
                'Email': email,
                'Phone': phone,
                'Address': address,
                'Links': links,
                'Experience': experience,
                'Education': education,
                'Languages': languages,
                'Certificates': certificates
            })

    # Convert to DataFrame and save as CSV
    df = pd.DataFrame(data)
    df.to_csv(output_csv_path, index=False)
    print(f'Data saved to {output_csv_path}')

# Example usage
process_resumes('C:\\Users\\santhoshs.s\\jupyter\\resumes\\data\\data\\BPO\\bb', 'output4.csv')


Data saved to output4.csv


In [5]:
!pip install pytesseract

In [9]:
import pytesseract
from PIL import Image

# Specify the path to your Tesseract executable
pytesseract.pytesseract.tesseract_cmd = r'C:\\Users\\santhoshs.s\\AppData\\Local\\Programs\\Python\\Python312\\Scripts\\tesseract.exe'  # Adjust this path if needed

# Load the image
image_path = 'C:\\Users\\santhoshs.s\\Pictures\\ss.png'  # Update with your image path
image = Image.open(image_path)

# Perform OCR on the image
extracted_text = pytesseract.image_to_string(image)

print(extracted_text)

TesseractNotFoundError: C:\\Users\\santhoshs.s\\AppData\\Local\\Programs\\Python\\Python312\\Scripts\\tesseract.exe is not installed or it's not in your PATH. See README file for more information.